全局管理器 \

用于管理全局变量（类） \
![关系图](./src/1.png) \

1. get_instance(name=’’, **kwargs)：创建或者返回对应名字的的实例。
2. get_current_instance()：返回最近被创建的实例。
3. instance_name：获取对应实例的 name。

使用

In [ ]:
#1.定义需要全局管理的类
from mmengine.utils import ManagerMixin

class GlobalClass(ManagerMixin):
    def __init__(self,name,value): #name是必选参数且需要传给父类
        super().__init__(name) 
        self.value=value

#2.创建实例对象，可以在任何地方，如Hook
from mmengine import Hook

class CustomHook(Hook):
    def before_run(self, runner):
        GlobalClass.get_instance('mmengine', value=50) #没有mmengine的GlobalClass实例，所以会创建
        GlobalClass.get_instance(runner.experiment_name, value=100)

#3.访问创建的对象，可以在任何地方，如Hook
import torch.nn as nn


class CustomModule(nn.Module):
    def forward(self, x):
        value = GlobalClass.get_current_instance().value  # 最近一次被创建的实例 value 为 100（步骤二中按顺序创建）
        value = GlobalClass.get_instance('mmengine').value  # 名为 mmengine 的实例 value 为 50
        # value = GlobalClass.get_instance('mmengine', 1000).value  # mmengine 已经被创建，不能再接受额外参数